In [ ]:
import geemap
import ee
from Monarch import user_gee

In [ ]:
Map = geemap.Map()
Map

In [ ]:
roi = geemap.geojson_to_ee(r'H:/Monarch/paper/Data/area.geojson').geometry()
Map.center_object(roi, 12)
Map.addLayer(roi, {}, 'roi')

In [ ]:
visParams = {'max': 3000, 'min': 0, 'bands': ['B4', 'B3', 'B2']}

In [ ]:
for year in range(2016, 2021):
    s2_year = []
    for i in range(3, 11):
        s2 = ee.ImageCollection("COPERNICUS/S2")\
                .filterBounds(roi)\
                .sort('CLOUD_COVERAGE_ASSESSMENT', False)\
                .filterDate(str(year), str(year+1))\
                .filter(ee.Filter.calendarRange(, i, 'day'))\
                .map(user_gee.rm_cloud_s2_sr)\
                .mosaic().clip(roi)
        s2_year.append(s2)
    s2_year = ee.ImageCollection(s2_year)
    geemap.download_ee_video(s2_year, visParams, f'H:/Monarch/paper/Data/S2_image_Thumbnail_git/S2_{year}.gif')